LP reformulation and 20 problems solved

In [5]:
import numpy as np
import pandas as pd

np.random.seed(42)

# === Custom Simplex Solver (Phase I + II) ===
class SimplexSolver:
    def __init__(self, A_eq, b, c):
        self.A = A_eq
        self.b = b
        self.c = c
        self.m, self.n = A_eq.shape

    def solve(self):
        # Phase I: Add artificial variables
        A_aug = np.hstack([self.A, np.eye(self.m)])
        c_phase1 = np.concatenate([np.zeros(self.n), np.ones(self.m)])
        tableau = np.zeros((self.m + 1, self.n + self.m + 1))
        tableau[:self.m, :self.n + self.m] = A_aug
        tableau[:self.m, -1] = self.b
        tableau[self.m, :self.n + self.m] = c_phase1

        basis = list(range(self.n, self.n + self.m))

        def pivot(tab, row, col):
            tab[row] = tab[row] / tab[row, col]
            for r in range(len(tab)):
                if r != row:
                    tab[r] -= tab[r, col] * tab[row]

        # Phase I iterations
        while True:
            col = np.argmin(tableau[self.m, :-1])
            if tableau[self.m, col] >= 0:
                break
            eligible = tableau[:self.m, col] > 1e-8
            if not np.any(eligible):
                raise Exception("Infeasible Phase I")
            ratios = tableau[:self.m, -1] / tableau[:self.m, col]
            ratios[~eligible] = np.inf
            row = np.argmin(ratios)
            pivot(tableau, row, col)
            basis[row] = col

        if tableau[self.m, -1] > 1e-8:
            raise Exception("Infeasible LP")

        # Phase II
        tableau = tableau[:self.m, :]
        c_ext = np.concatenate([self.c, np.zeros(self.m)])
        cost_row = np.zeros_like(tableau[0])
        for i, bi in enumerate(basis):
            cost_row -= c_ext[bi] * tableau[i]
        cost_row[:-1] += c_ext
        tableau = np.vstack([tableau, cost_row])

        while True:
            col = np.argmin(tableau[-1, :-1])
            if tableau[-1, col] >= -1e-10:
                break
            eligible = tableau[:-1, col] > 1e-8
            if not np.any(eligible):
                raise Exception("Unbounded LP")
            ratios = tableau[:-1, -1] / tableau[:-1, col]
            ratios[~eligible] = np.inf
            row = np.argmin(ratios)
            pivot(tableau, row, col)
            basis[row] = col

        x = np.zeros(self.n + self.m)
        for i, bi in enumerate(basis):
            if bi < self.n + self.m:
                x[bi] = tableau[i, -1]
        return x[:self.n], tableau[-1, -1]

def build_lp_problem(c):
    n = len(c)
    c_reform = np.concatenate([c, -c])
    A = np.ones((1, 2 * n))
    b = np.array([1.0])
    A_eq = np.hstack([A, np.eye(1)])  # Add slack var
    c_full = np.concatenate([c_reform, np.zeros(1)])
    return A_eq, b, c_full

def solve_lp_custom_simplex(c):
    n = len(c)
    c_reform = np.concatenate([c, -c])
    A = np.ones((1, 2 * n))
    b = np.array([1.0])
    A_eq = np.hstack([A, np.eye(1)])  # Add slack var
    c_full = np.concatenate([c_reform, np.zeros(1)])
    solver = SimplexSolver(A_eq, b, c_full)
    x_full, fval = solver.solve()
    x_plus = x_full[:n]
    x_minus = x_full[n:2*n]
    x = x_plus - x_minus
    return x, fval

# === Batch Runner and Results Table ===
import time

def run_all_tests_custom():
    dims = [2, 3, 4, 5, 10, 20, 30, 50, 100, 200]
    results = []
    for n in dims:
        for x0 in [np.zeros(n), np.ones(n)]:
            c = np.random.randn(n)
            start_time = time.time()
            try:
                x_lp, f_lp = solve_lp_custom_simplex(c)
                success = True
                error_msg = ""
                norm_l1 = np.linalg.norm(x_lp, 1)
                norm_l2 = np.linalg.norm(x_lp, 2)
            except Exception as e:
                success = False
                error_msg = str(e)
                x_lp = None
                f_lp = None
                norm_l1 = None
                norm_l2 = None
            end_time = time.time()
            duration = end_time - start_time

            results.append({
                'n': n,
                'x0': 'zero' if np.all(x0 == 0) else 'ones',
                'LP_val': f_lp,
                'success': success,
                'error_msg': error_msg,
                'solution_norm_l1': norm_l1,
                'solution_norm_l2': norm_l2,
                'time_sec': duration,
                'solution_vector': x_lp
            })
    return pd.DataFrame(results)


# === Run Everything ===
df = run_all_tests_custom()
print(df)


      n    x0    LP_val  success error_msg  solution_norm_l1  \
0     2  zero  0.496714     True                         1.0   
1     2  ones  1.523030     True                         1.0   
2     3  zero  1.579213     True                         1.0   
3     3  ones  0.767435     True                         1.0   
4     4  zero  1.913280     True                         1.0   
5     4  ones  1.724918     True                         1.0   
6     5  zero  1.465649     True                         1.0   
7     5  ones  1.424748     True                         1.0   
8    10  zero  1.959670     True                         1.0   
9    10  ones  1.478522     True                         1.0   
10   20  zero  1.763040     True                         1.0   
11   20  ones  2.619745     True                         1.0   
12   30  zero  2.463242     True                         1.0   
13   30  ones  2.190456     True                         1.0   
14   50  zero  2.720169     True        